In [1]:
import uproot
import numpy as np
import numba
import awkward1 as ak
import os
import optparse

from data import getData
from plot_utils import plotHist, plotCollection, plotEfficiency, combinePDFs
from truth_utils import truth_link, dr_match

In [2]:
cms_events = getData()

In [3]:
truth_builder = truth_link(cms_events['genParticles'], ak.ArrayBuilder())
truth_extension = truth_builder.snapshot()
ele_mask = np.abs(cms_events['genParticles']['pdgId']) == 11
first_mask = truth_extension['isFirst']
last_mask = np.abs(cms_events['genParticles']['status']) == 1 # equivalent to last                                                                                                            
z_mask = truth_extension['motherPdgId'] == 23
n2_mask = truth_extension['motherPdgId'] == 1000023
gen_ele_mask = ele_mask & (z_mask | n2_mask) & last_mask
truth_electrons = cms_events['genParticles'][gen_ele_mask]

In [4]:
reco_electrons = cms_events['softElectrons']
#reco_electrons = cms_events['electrons']
reco_cuts = {
    "all": reco_electrons.pt > -1,
#    "looseMVA": reco_electrons.mvaId > 0,
#    "tightMVA": reco_electrons.mvaId > 5,
    }


In [5]:
cut_name="all"
reco_mask = reco_cuts[cut_name]
signal_electrons = reco_electrons[reco_mask]

match_builder = dr_match(truth_electrons, signal_electrons, ak.ArrayBuilder(), doReco=False)
match_extension = match_builder.snapshot()
matching_truth_mask = match_extension.truth_to_reco_index >= 0
# matching_reco_mask = match_extension.truth_to_reco_index[matching_truth_mask]                                                                                                      

match_builderR = dr_match(truth_electrons, signal_electrons, ak.ArrayBuilder(), doReco=True)
match_extensionR = match_builderR.snapshot()
matching_reco_mask = match_extensionR.reco_to_truth_index >= 0
nonmatching_reco_mask = match_extensionR.reco_to_truth_index < 0

matched_truth = truth_electrons[matching_truth_mask]
unmatched_truth = truth_electrons[~matching_truth_mask]
matched_reco = signal_electrons[matching_reco_mask]
unmatched_reco = signal_electrons[~matching_reco_mask]
# unmatched_reco = signal_electrons[nonmatching_reco_mask]      

In [34]:
plotCollection([matched_reco,unmatched_reco], cut_name+"_matching_reco", leg=["matched","unmatched"],
                           outDir="output_plots/diagnostic/match_comparison/reco_"+cut_name, normAttrs=False)

Saved: output_plots/diagnostic/match_comparison/reco_all/n_all_matching_reco.pdf
Saved: output_plots/diagnostic/match_comparison/reco_all/all_matching_reco_pt.pdf
Saved: output_plots/diagnostic/match_comparison/reco_all/all_matching_reco_phi.pdf
Saved: output_plots/diagnostic/match_comparison/reco_all/all_matching_reco_eta.pdf
Saved: output_plots/diagnostic/match_comparison/reco_all/all_matching_reco_mass.pdf
Saved: output_plots/diagnostic/match_comparison/reco_all/all_matching_reco_dxy.pdf
Saved: output_plots/diagnostic/match_comparison/reco_all/all_matching_reco_dxyErr.pdf
Saved: output_plots/diagnostic/match_comparison/reco_all/all_matching_reco_dz.pdf
Saved: output_plots/diagnostic/match_comparison/reco_all/all_matching_reco_dzErr.pdf
Saved: output_plots/diagnostic/match_comparison/reco_all/all_matching_reco_fBrem.pdf
Saved: output_plots/diagnostic/match_comparison/reco_all/all_matching_reco_ip3d.pdf
Saved: output_plots/diagnostic/match_comparison/reco_all/all_matching_reco_mvaId.p

In [17]:
attr='pt'
objs = [matched_reco,unmatched_reco]
vals = [ak.to_list(ak.flatten(o[attr])) for o in objs]
#plotHist('test', var=attr, vals=ak.flatten(objs[attr]), outDir='.', norm=True)
x = plotHist('test_'+attr, var=attr, vals=vals)

Saved: plots/test_pt.pdf


In [37]:
#help(matched_reco)

In [33]:
#reco_electrons[reco_electrons.pt > -1][0].pt
#matching_reco_mask = match_extensionR.reco_to_truth_index >= 0
#ak.to_list(matching_reco_mask[0])
print(matched_reco.pt[0:5])
print(ak.num(unmatched_reco.pt)[0:5])

[[2.82], [], [5.05], [], []]
[75, 48, 89, 70, 52]


In [9]:
nprint=10
print(ak.num(truth_electrons)[:nprint])
print(ak.num(reco_electrons)[:nprint])
print(ak.to_list(truth_electrons.eta)[:nprint])
print(ak.to_list(reco_electrons.eta)[:nprint])
print(ak.to_list(truth_electrons.phi)[:nprint])
print(ak.to_list(reco_electrons.phi)[:nprint])

[2, 0, 2, 0, 0, 2, 0, 2, 0, 2]
[76, 48, 90, 70, 52, 45, 106, 34, 78, 53]
[[0.4287109375, -3.265625], [], [-1.88671875, -1.625], [], [], [0.353515625, -0.7578125], [], [-1.8203125, -1.70703125], [], [-0.21044921875, -1.359375]]
[[0.0011305809020996094, -0.0026144981384277344, -0.021240234375, -0.0868988037109375, -0.22894287109375, 1.20751953125, 0.35430908203125, 0.0021147727966308594, -0.28009033203125, -0.236602783203125, 1.15234375, -1.387939453125, 0.4271240234375, -0.071014404296875, -0.8541259765625, -0.8052978515625, -0.0077037811279296875, -0.0011441707611083984, -0.43572998046875, -1.06201171875, 1.697021484375, -0.9488525390625, -0.669677734375, -0.9461669921875, -0.4658203125, -0.876953125, -1.306884765625, 0.8787841796875, 2.4150390625, -0.90771484375, 1.893310546875, -0.20928955078125, 1.892822265625, -0.183074951171875, 0.2813720703125, -0.146148681640625, -0.128631591796875, -2.49072265625, -0.76123046875, -0.44677734375, 1.0908203125, -0.0043544769287109375, 0.275817871

[[3.03076171875, 3.0244140625, 0.170074462890625, 0.22576904296875, 0.0172119140625, -3.12646484375, -2.94091796875, 0.057708740234375, 0.20526123046875, 0.3795166015625, -3.017578125, -2.689453125, -2.9091796875, 0.45220947265625, 2.29296875, -0.074310302734375, 0.2423095703125, 2.52734375, -2.0908203125, -3.0458984375, -1.96826171875, 1.367431640625, 0.528564453125, -0.05591583251953125, 0.918212890625, 0.1154327392578125, -0.235687255859375, 0.8931884765625, -1.6494140625, -1.260986328125, 0.8828125, 1.62939453125, 0.27740478515625, 0.187652587890625, -2.61572265625, 2.02294921875, 0.0815582275390625, -0.88671875, -0.798828125, -2.71533203125, -3.09814453125, -0.716552734375, -1.05712890625, 1.7353515625, -1.790771484375, -0.9661865234375, 2.744140625, -2.228515625, 0.25384521484375, -1.982177734375, -0.3687744140625, 2.71435546875, 2.84130859375, -1.82421875, 1.046142578125, -1.302490234375, -1.291748046875, -2.96240234375, -2.486328125, 2.84716796875, -2.466796875, -1.7744140625, 

In [10]:
print(ak.num(matched_truth.pt)[:nprint])
print(ak.to_list(matched_truth.pt)[:nprint])

[1, 0, 1, 0, 0, 1, 0, 2, 0, 2]
[[2.8671875], [], [4.96875], [], [], [10.09375], [], [3.4140625, 6.984375], [], [9.46875, 2.96875]]


In [11]:
print(ak.num(unmatched_truth.pt)[:nprint])
print(ak.to_list(unmatched_truth.pt)[:nprint])

[1, 0, 1, 0, 0, 1, 0, 0, 0, 0]
[[0.74609375], [], [2.265625], [], [], [1.453125], [], [], [], []]


In [12]:
print(ak.num(matched_reco.pt)[:nprint])
print(ak.to_list(matched_reco.pt)[:nprint])

[1, 0, 1, 0, 0, 1, 0, 2, 0, 2]
[[2.8238096237182617], [], [5.0460638999938965], [], [], [10.34103012084961], [], [8.194936752319336, 2.993222236633301], [], [9.54413890838623, 2.9216201305389404]]


In [13]:
print(ak.num(unmatched_reco.pt)[:nprint])
print(ak.to_list(unmatched_reco.pt)[:nprint])

[75, 48, 89, 70, 52, 44, 106, 32, 78, 51]
[[41.665260314941406, 41.2654914855957, 10.652962684631348, 6.707527160644531, 5.121514797210693, 4.767570972442627, 4.389225959777832, 4.333414077758789, 4.266343593597412, 3.866128921508789, 3.535625696182251, 2.864638566970825, 2.3002939224243164, 2.165043592453003, 2.064357042312622, 1.8168129920959473, 1.402222752571106, 1.3537236452102661, 1.351630449295044, 1.3481934070587158, 1.3275318145751953, 1.241409182548523, 1.1958914995193481, 1.1944098472595215, 1.1823455095291138, 1.0589981079101562, 1.050458312034607, 1.047339916229248, 1.0371196269989014, 1.0190353393554688, 1.0175362825393677, 1.015115737915039, 0.9829141497612, 0.9584792852401733, 0.9478253722190857, 0.9465541839599609, 0.9072566628456116, 0.8783331513404846, 0.871503472328186, 0.8319242000579834, 0.8271585702896118, 0.8223638534545898, 0.8183886408805847, 0.8115609884262085, 0.8030329346656799, 0.7904025316238403, 0.7876651287078857, 0.7872351408004761, 0.7870201468467712,

In [14]:
print((ak.num(matched_truth.pt)+ak.num(unmatched_truth.pt))[:nprint])
print(ak.num(truth_electrons)[:nprint])

[2, 0, 2, 0, 0, 2, 0, 2, 0, 2]
[2, 0, 2, 0, 0, 2, 0, 2, 0, 2]


In [15]:
print((ak.num(matched_reco.pt)+ak.num(unmatched_reco.pt))[:nprint])
print(ak.num(reco_electrons)[:nprint])

[76, 48, 90, 70, 52, 45, 106, 34, 78, 53]
[76, 48, 90, 70, 52, 45, 106, 34, 78, 53]


Now check that the contents are OK

In [16]:
print((matched_truth.pt)[:nprint])
print((unmatched_truth.pt)[:nprint])
print((truth_electrons.pt)[:nprint])

[[2.87], [], [4.97], [], [], [10.1], [], [3.41, 6.98], [], [9.47, 2.97]]
[[0.746], [], [2.27], [], [], [1.45], [], [], [], []]
[[2.87, 0.746], [], [4.97, 2.27], [], [], ... [], [3.41, 6.98], [], [9.47, 2.97]]


In [35]:
x = plotHist('test', vals=[[1,1,2,3,4],[5,5,5,6,7,8]])

Saved: plots/test.pdf
